<a href="https://colab.research.google.com/github/AllKashirin/Education/blob/main/Set_cell_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install transliterate

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import pandas as pd
import numpy as np
import re
from transliterate import translit
import math
import time
import os
from tqdm import tqdm
from datetime import datetime
from openpyxl import load_workbook

In [57]:
def split_cells(cell):
    pattern = r'\W'
    return [x for x in re.split(pattern, cell) if x]

In [58]:
def split_symbols(column):
    all_data = column['namen'].split()
    all_data = [split_cells(z) for z in all_data]
    all_data = [item for data_list in all_data for item in data_list]
    letters = [item for item in all_data if item.isalpha()]
    digits = [item for item in all_data if item.isdigit()]
    return pd.Series({'namen': all_data, 'Модель': letters, 'Модификация': digits})

In [59]:
#def set_only_name(column):
 #   flag=false
  #  used_names=[]
   #     for row in column.itertuples():
    #    namen=''
     #   if name in Cars_names        #(item.isalpha() and len(item)>2) and (item in Cars_names):
      #      continue                 #name = item
       # model = row['Модель']        #flag = true
                                     #break
        #elif [item for item in mod_data if (item.isalpha() and len(item)>2) and (item in Cars_names)]:
         #   name = item
    #return pd.Series({'Марка =': name})

In [81]:
def clean_rows(data):
    #data = data.drop(axis=0, index = [0, 1, 2])
    #очищаем столбец Название = замена всего лишнего на пробел один!
    for column in ['namen','Марка','Модель','Модификация']:
        data[column] = data[column].astype(str)
        data[column] = data[column].str.lower()
        data[column] = data[column].str.replace(r'[^а-яa-z0-9\s-]', '', regex = True)
        data[column] = data[column].str.replace(r'-+', ' ', regex = True)
        data[column] = data[column].str.replace(r'\s{2,}', ' ', regex = True)
        data[column] = data[column].str.strip()
    #формируем наименование марки по первому вхождению названия в столбец "namen"
    #data['namen'] = data.apply(set_only_name, axis = 1)
    #print(data[column])
    return data

In [83]:
def rename(data):
    data = data.drop(axis=0, index = [0, 1, 2])
    stolbnm = []
    stolbmdl = []
    index = 0
    for row in data.itertuples():
        letter1 = row[4].split()
        letter2 = row[5].split()
        name = [item for item in letter1 if (item.isalpha() and len(item)>2) and (item in Cars_names)]
        if (name == '')|(name==None):
            name = 'not defined'
        model = [item for item in letter2 if (item.isalpha() and len(item)>2) and (item in Cars_names)]
        if (model == '')|(model==None):
            model = 'not defined'
        print(row)
        index=row[0]
        stolbnm [0] = stolbnm.append(name)
      #  if (stolbnm[index] == '')|(stolbnm[index]==None):
    #     stolbnm[index] = model
        stolbmdl[0] = stolbmdl.append(model)
        print('Столбец марка = ', index, stolbnm)
        print('Столбец модель = ', index, stolbmdl)
        print('------------------------------------')
    return pd.Series({stolbnm})

In [62]:
def set_cell_value(cell_value, template):
    if cell_value.startswith(template):
        return template
    else:
        return cell_value

In [63]:
#data['Марка']=data['Марка'].apply(lambda x: set_cell_value(x,template_value))

In [64]:
df = pd.read_excel(io='neucht.xlsx', sheet_name='NGHK', engine='openpyxl')
df.head()

,№ п/п,Категория техники,Марка,Модель,Модификация
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,1.0,2,3,4,5.0
3,2.0,ДСТ,КРАН ДЕМАГ ТС - 2600 /500T/,NaN,NaN
4,3.0,ДСТ,ДЭК-631 КС-7161 А,NaN,NaN


# считываем в датафрейм перечень - шаблоны наименований единиц техники

In [65]:
list_of_cars = pd.read_excel(io='Spissokk.xlsx', sheet_name='list1', engine='openpyxl')
list_of_cars.head()

,Марка,Модель,Модификация
0,Akerman,UMG,EC-200
1,Alphard,NaN,NaN
2,Antworker,805,NaN
3,Arctic,1800,NaN
4,Arctic,Cat SNO 600 PRO,NaN


In [66]:
list_of_cars['Марка'] = list_of_cars['Марка'].astype(str)
list_of_cars['Марка'] = list_of_cars['Марка'].str.lower()

формируем список уникальных наименований = шаблонов

In [67]:
temp=list_of_cars['Марка'].tolist()
Cars_names=[]
[Cars_names.append(x) for x in temp if x not in Cars_names]
print(Cars_names)

['akerman', 'alphard', 'antworker', 'arctic', 'argo', 'argo-avenger', 'atlas copco', 'atlet', 'audi', 'aurus', 'autocut', 'avant', 'balkancar', 'bandvagn', 'baumaster', 'baw', 'bearcat', 'bearford', 'bmw', 'bobcat', 'bombardier', 'bulcar', 'carver', 'caterpillar', 'challenger', 'chevrolet', 'claeys', 'cummins', 'daewoo', 'demag', 'detvan', 'dieci', 'digger', 'doosan', 'dressta', 'elaz', 'elitech', 'endress', 'energo', 'eurokar', 'fablok', 'fastrac', 'faun', 'fd25с6', 'ferri', 'fiat', 'fiatallis', 'fobos', 'ford', 'forpost', 'forward', 'foxweld', 'freightliner', 'fresia ', 'geka', 'genesis', 'gesan', 'gmc', 'great wall ', 'grove', 'gys', 'hako', 'hako-citymaster ', 'hasky', 'heli', 'highlander', 'hitachi', 'honda', 'husgvama', 'husgvana', 'husky', 'husqvarna', 'huster', 'hyundai', 'ikarbus', 'indos', 'isuzu', 'iveco', 'iveco-amt', 'jac', 'jcb', 'jeep', 'john deere', 'kalmar', 'kamatsu', 'kamaz', 'kato', 'kawasaki', 'kia', 'kiitokori', 'king long', 'komatsu', 'kubota', 'lada', 'land rove

In [68]:
trrus_cars_names = list_of_cars

транслитерируем шаблоны наименований

In [69]:
trrus_cars_names['Марка']=list_of_cars.apply(lambda x: translit(x['Марка'], 'ru'), axis=1)

добавляем транслитерируемые наименования к исходному списку шаблонов

In [70]:
_=trrus_cars_names['Марка'].tolist()
[Cars_names.append(x) for x in _ if x not in Cars_names]
print(Cars_names)

['akerman', 'alphard', 'antworker', 'arctic', 'argo', 'argo-avenger', 'atlas copco', 'atlet', 'audi', 'aurus', 'autocut', 'avant', 'balkancar', 'bandvagn', 'baumaster', 'baw', 'bearcat', 'bearford', 'bmw', 'bobcat', 'bombardier', 'bulcar', 'carver', 'caterpillar', 'challenger', 'chevrolet', 'claeys', 'cummins', 'daewoo', 'demag', 'detvan', 'dieci', 'digger', 'doosan', 'dressta', 'elaz', 'elitech', 'endress', 'energo', 'eurokar', 'fablok', 'fastrac', 'faun', 'fd25с6', 'ferri', 'fiat', 'fiatallis', 'fobos', 'ford', 'forpost', 'forward', 'foxweld', 'freightliner', 'fresia ', 'geka', 'genesis', 'gesan', 'gmc', 'great wall ', 'grove', 'gys', 'hako', 'hako-citymaster ', 'hasky', 'heli', 'highlander', 'hitachi', 'honda', 'husgvama', 'husgvana', 'husky', 'husqvarna', 'huster', 'hyundai', 'ikarbus', 'indos', 'isuzu', 'iveco', 'iveco-amt', 'jac', 'jcb', 'jeep', 'john deere', 'kalmar', 'kamatsu', 'kamaz', 'kato', 'kawasaki', 'kia', 'kiitokori', 'king long', 'komatsu', 'kubota', 'lada', 'land rove

этот столбец надо рассортировать

In [71]:
new_column = df['Марка'].astype(str)
new_column

0                                                   nan
1                                                   nan
2                                                     3
3                           КРАН ДЕМАГ ТС - 2600 /500T/
4                                    ДЭК-631 КС-7161 А 
5                                    MANITOU M 50 - 4 T
6                  Снегоочиститель СШР-1 мод. 003-СА-В1
7                  Машина уборочная универсальная 54852
8     Погрузчик вилочный СО, Combilift C4000  г/п - ...
9                                   TOYOTA LAND CRUISER
10                                    NRCTZB   АГП 2204
11                                     IVECO-АМТ 653901
12    Кран автомобильный КС-65719-5К на шасси КАМАЗ ...
13                                               32591N
14                                395406 (ISUZU NQR75R)
15                        КО-530-05 на шасси МАЗ 6303А5
16                                АГП-32РТ марка 5328ВН
17                                   ГАЗель Next

добавили столбец namen - чтобы оставить 3 результирующих столбца = марка, модель, модификация

In [77]:
df3=df
df3.head()

,№ п/п,Категория техники,namen,Марка,Модель,Модификация
0,NaN,NaN,nan,NaN,NaN,NaN
1,NaN,NaN,nan,NaN,NaN,NaN
2,1.0,2,3,3,4,5.0
3,2.0,ДСТ,КРАН ДЕМАГ ТС - 2600 /500T/,КРАН ДЕМАГ ТС - 2600 /500T/,NaN,NaN
4,3.0,ДСТ,ДЭК-631 КС-7161 А,ДЭК-631 КС-7161 А,NaN,NaN


In [78]:
#df3 = df.insert( 2 , 'namen', new_column)

разбиваю каждую ячейку на группы букв и цифр

In [79]:
clean_rows(df3)

,№ п/п,Категория техники,namen,Марка,Модель,Модификация
0,NaN,NaN,nan,nan,nan,nan
1,NaN,NaN,nan,nan,nan,nan
2,1.0,2,3,3,4,50
3,2.0,ДСТ,кран демаг тс 2600 500t,кран демаг тс 2600 500t,nan,nan
4,3.0,ДСТ,дэк 631 кс 7161 а,дэк 631 кс 7161 а,nan,nan
5,4.0,ДСТ,manitou m 50 4 t,manitou m 50 4 t,nan,nan
6,5.0,СТ,снегоочиститель сшр 1 мод 003 са в1,снегоочиститель сшр 1 мод 003 са в1,шасси камаз 43118,nan
7,6.0,СТ,машина уборочная универсальная 54852,машина уборочная универсальная 54852,шасси урал 5557,nan
8,7.0,ДСТ,погрузчик вилочный со combilift c4000 гп 4 тн ...,погрузчик вилочный со combilift c4000 гп 4 тн ...,nan,nan
9,8.0,АТ,toyota land cruiser,toyota land cruiser,nan,nan


формирую два списка уникальных наименований для дальнейшего слияния в случае, когда "марка" не опознана из исходного столбца - надо взять инфу из "модели"

In [84]:
df4=rename(df3)

Pandas(Index=3, _1=2.0, _2='ДСТ ', namen='кран демаг тс 2600 500t', Марка='кран демаг тс 2600 500t', Модель='nan', Модификация='nan')
Столбец марка =  3 [None]
Столбец модель =  3 [None]
------------------------------------
Pandas(Index=4, _1=3.0, _2='ДСТ ', namen='дэк 631 кс 7161 а', Марка='дэк 631 кс 7161 а', Модель='nan', Модификация='nan')
Столбец марка =  4 [None, []]
Столбец модель =  4 [None, []]
------------------------------------
Pandas(Index=5, _1=4.0, _2='ДСТ ', namen='manitou m 50 4 t', Марка='manitou m 50 4 t', Модель='nan', Модификация='nan')
Столбец марка =  5 [None, [], ['manitou']]
Столбец модель =  5 [None, [], []]
------------------------------------
Pandas(Index=6, _1=5.0, _2='СТ', namen='снегоочиститель сшр 1 мод 003 са в1', Марка='снегоочиститель сшр 1 мод 003 са в1', Модель='шасси камаз 43118', Модификация='nan')
Столбец марка =  6 [None, [], ['manitou'], []]
Столбец модель =  6 [None, [], [], ['камаз']]
------------------------------------
Pandas(Index=7, _1=6.

TypeError: unhashable type: 'list'

In [ ]:
stolbnm

In [ ]:
stolbmdl